In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from awpy.analytics.nav import find_closest_area
import os
#from pyunpack import Archive
import json
import pandas as pd
from cs_bahtml.filtering import points_in_polygon, build_polygon
from cs_bahtml.gospa import calculate_gospa_distance

In [4]:
wd = os.path.dirname(os.getcwd())
data_dir = "..\\..\\csgo_demos"
map_dir = os.path.join(wd,"media","maps")

In [5]:
files = os.listdir(data_dir)
data_list = []
for file in files:
    path = os.path.join(data_dir,file)
    if not (('mirage' in path) & ('vitality' in path)):
        continue
    print(path)
    try:
        with open(path,'r') as f:
            data = json.load(f)
        df = pd.DataFrame.from_dict(data["gameRounds"]).loc[:,["roundNum","tTeam","ctTeam","grenades","frames"]]
        df['matchID'] = data["matchID"]
        df['mapName'] = data['mapName']
        print(df.loc[0,'ctTeam'])
        print(df.loc[0,'tTeam'])
        data_list.append(df)
    except:
        True
df = pd.concat(data_list)

..\..\csgo_demos\big-vs-vitality-m1-mirage_2363193.json
Vitality
BIG
..\..\csgo_demos\g2-vs-vitality-m1-mirage_2363197.json
Vitality
G2 Esports
..\..\csgo_demos\heroic-vs-vitality-m2-mirage_2364753.json
Vitality
Heroic
..\..\csgo_demos\monte-vs-vitality-m2-mirage_2363198.json
Vitality
Monte
..\..\csgo_demos\og-vs-vitality-m1-mirage_2363562.json
Team Vitality
OG
..\..\csgo_demos\vitality-vs-apeks-m1-mirage_2364291.json
Vitality
Apeks
..\..\csgo_demos\vitality-vs-big-m2-mirage_2363563.json
Team Vitality
BIG
..\..\csgo_demos\vitality-vs-faze-m2-mirage_2364750.json
Vitality
FaZe Clan
..\..\csgo_demos\vitality-vs-ninjas-in-pyjamas-m1-mirage.json
Vitality
Ninjas in Pyjamas


In [6]:
def clean_data(df,maximum_roundtime,rotate_seconds, min_observations):
    df.loc[:,'grenades'] = df['grenades'].apply(lambda gs: [g for g in gs if (
        (g['grenadeType']=='Smoke Grenade') | 
        (g['grenadeType'] == 'Molotov') | 
        (g['grenadeType'] == 'Incendiary Grenade')
        )
        ])
    df = df.explode('grenades').dropna().reset_index()
    df = pd.concat([df.drop('grenades',axis=1),pd.json_normalize(df['grenades'])],axis=1)
    df = df[df['throwSeconds']<= maximum_roundtime]
    df['throwerSide'] = df['throwerSide'].apply(lambda side: side.lower())
    df['opponentSide'] = df['throwerSide'].apply(lambda side: "t" if side =="ct" else "ct")
    df.loc[df['throwerTeam']=="Team Vitality", 'throwerTeam'] = 'Vitality'
    df.loc[df['grenadeType']=='Incendiary Grenade', 'grenadeType'] = 'Molotov'
    df =  df[df['throwerTeam'] == "Vitality"]
    df['opponentName'] = df.apply(lambda row: row.frames[0][row['opponentSide']]['teamName'],axis=1)
    df['throwCords'] = df.apply(lambda row: (row['throwerX'],row['throwerY'],row['throwerZ']), axis = 1)
    df['landCords'] = df.apply(lambda row: (row['grenadeX'],row['grenadeY'],row['grenadeZ']), axis = 1)
    df['throwArea'] = df.apply(lambda row: find_closest_area(row['mapName'], row['throwCords'])['areaId'], axis= 1)
    df['landArea'] = df.apply(lambda row: find_closest_area(row['mapName'], row['landCords'])['areaId'], axis = 1)
    df['players'] = df.apply(lambda row: [frame[row['throwerSide']]['players'] for frame in row.frames if frame['tick']>=row.throwTick+rotate_seconds*128 if frame['tick']<row.throwTick+128+rotate_seconds*128],axis=1)
    df = df[df['players'].apply(len)>0]
    df['dead_players'] = df['players'].apply(lambda players: [(player['x'],player['y'],player['z']) for player in players[0] if player['isAlive']==False])
    df['players'] = df['players'].apply(lambda players: [(player['x'],player['y'],player['z']) for player in players[0] if player['isAlive']==True])
    df_players = df.groupby(["mapName","throwerSide","throwArea","landArea","grenadeType"])['players'].apply(list).reset_index()
    df_players = df_players[df_players['players'].apply(len)>=min_observations]
    df_players['util_score'] = df_players.apply(lambda row: calculate_gospa_distance(row['players'],c = 2000, p = 1, alpha = 2, map=row['mapName']), axis = 1)
    df_players = df_players.loc[:, ["mapName","throwerSide","throwArea","landArea","grenadeType","util_score"]]
    df = df.loc[:, ['matchID', 'mapName',"throwerSide","throwerName","opponentName",'throwSeconds','grenadeType','throwArea','landArea', 'dead_players', 'players', 'throwCords','landCords']]
    df = df.merge(df_players, how = "inner", on = ["mapName","throwerSide","throwArea","landArea","grenadeType"])
    
    return df

In [7]:
df2 = clean_data(df,30,15, 3)

1103.3102976608086
1056.3822131412678
888.6689151064817
1357.1343914515971
1065.0716627793788
1246.7637380188478
944.6588681667746
1311.4036752240736
1150.6724666506054
1053.6713049750654
1241.7452810052018
965.9711485245294
1154.7016625106571
1108.9374807513664
1133.8671958683433
999.1650595949764
1177.3228435523956
1033.6902513430991
1211.3879343467192
1230.765486218364
1298.7982749351079
1091.7297322857512
2248.4176433732355
1159.6517831998547
1036.339604971908
1159.894268958852
1180.888554256713
932.7008001921573
935.3077877735894
986.0455338026223
1110.4862686260853
1265.4082222466848
881.2106124867356
934.3533437650325
1440.174057515895
1170.3203957315739
956.4762095726181
1094.1779915244344
1142.2027400209631
1014.5295162530186
1214.092535415463
1217.7659565154188
1269.9648010991045
1520.8058416832878
862.9633413108759
1220.037352029516
1476.885115171519
1298.971489230721
1131.5446762319496
1869.8347828607045
1891.355037524351
1118.3439424747999


In [92]:
df2['util_score'].min()

588.2457006726031

In [93]:
df2

matchID    mapName throwerSide  \
0          big-vs-vitality-m1-mirage_2363193  de_mirage          ct   
1          big-vs-vitality-m1-mirage_2363193  de_mirage          ct   
2        monte-vs-vitality-m2-mirage_2363198  de_mirage          ct   
3           og-vs-vitality-m1-mirage_2363562  de_mirage          ct   
4           og-vs-vitality-m1-mirage_2363562  de_mirage          ct   
..                                       ...        ...         ...   
428       vitality-vs-faze-m2-mirage_2364750  de_mirage          ct   
429  vitality-vs-ninjas-in-pyjamas-m1-mirage  de_mirage          ct   
430  vitality-vs-ninjas-in-pyjamas-m1-mirage  de_mirage          ct   
431  vitality-vs-ninjas-in-pyjamas-m1-mirage  de_mirage          ct   
432  vitality-vs-ninjas-in-pyjamas-m1-mirage  de_mirage          ct   

          opponentName  throwSeconds    grenadeType  throwArea  landArea  \
0                  BIG      4.929688  Smoke Grenade        119       456   
1                  BIG     25.070312  Smoke Grenade        119       456   
2                Monte     24.984375  Smoke Grenade        119       456   
3                   OG     16.796875  Smoke Grenade        119       456   
4                   OG      6.234375  Smoke Grenade        119       456   
..                 ...           ...            ...        ...       ...   
428          FaZe Clan      6.140625        Molotov        232       804   
429  Ninjas in Pyjamas      6.921875        Molotov        232       804   
430  Ninjas in Pyjamas     11.796875        Molotov         35       373   
431  Ninjas in Pyjamas     12.156250        Molotov         35       373   
432  Ninjas in Pyjamas     12.164062        Molotov         35       373   

                                          dead_players  \
0                                                   []   
1    [(-1376.7142333984375, 165.16455078125, -165.2...   
2                                                   []   
3                                                   []   
4                                                   []   
..                                                 ...   
428                                                 []   
429                                                 []   
430  [(-762.1006469726562, -2018.250244140625, -179...   
431  [(-159.89833068847656, -583.015380859375, -229...   
432                                                 []   

                                               players  \
0    [(105.35396575927734, -2009.28125, -167.96875)...   
1    [(-2518.768310546875, 152.94439697265625, -166...   
2    [(-2119.762451171875, 719.7903442382812, -95.9...   
3    [(-1277.3092041015625, -969.25634765625, -167....   
4    [(-1216.084228515625, -577.38525390625, -167.9...   
..                                                 ...   
428  [(-2367.84326171875, 711.14794921875, -39.9687...   
429  [(-2186.748779296875, 814.685302734375, -124.1...   
430  [(-381.2843933105469, -522.655517578125, -139....   
431  [(-821.4595947265625, -218.50201416015625, -16...   
432  [(-897.6497192382812, -751.6567993164062, -263...   

                                throwCords  \
0     (-761.9375, -2155.84375, -113.28125)   
1    (-795.46875, -2211.46875, -112.78125)   
2        (-793.34375, -2210.96875, -112.5)   
3       (-761.5625, -2194.40625, -112.625)   
4       (-728.25, -2145.09375, -113.03125)   
..                                     ...   
428     (-689.125, -2201.8125, -112.34375)   
429     (-706.15625, -2213.71875, -112.25)   
430     (-2018.71875, 410.6875, -89.40625)   
431             (-2033.0, 386.5, -89.4375)   
432         (-2012.28125, 418.0, -89.3125)   

                                landCords   util_score  
0    (-597.34375, -897.09375, -245.96875)  1241.745281  
1     (-595.40625, -868.8125, -261.96875)  1241.745281  
2          (-603.8125, -825.4375, -262.0)  1241.745281  
3    (-594.59375, -883.71875, -253.96875)  1241.745281  
4      (-594.625, -877.84375, -253.9687

In [81]:
groups = df2.groupby(["mapName","throwerSide","throwArea","landArea","grenadeType"]).groups
group_list = [(df2.loc[index,:],df2.loc[index[0],"util_score"]) for index in groups.values()]
sorted_list = [sort[0] for sort in sorted(group_list, key = lambda tup: tup[1])]
sorted_list[0]

matchID    mapName throwerSide opponentName  \
322     g2-vs-vitality-m1-mirage_2363197  de_mirage           t   G2 Esports   
323     g2-vs-vitality-m1-mirage_2363197  de_mirage           t   G2 Esports   
324     og-vs-vitality-m1-mirage_2363562  de_mirage           t           OG   
325     og-vs-vitality-m1-mirage_2363562  de_mirage           t           OG   
326     og-vs-vitality-m1-mirage_2363562  de_mirage           t           OG   
327  vitality-vs-apeks-m1-mirage_2364291  de_mirage           t        Apeks   
328  vitality-vs-apeks-m1-mirage_2364291  de_mirage           t        Apeks   
329    vitality-vs-big-m2-mirage_2363563  de_mirage           t          BIG   
330   vitality-vs-faze-m2-mirage_2364750  de_mirage           t    FaZe Clan   

     throwSeconds    grenadeType  throwArea  landArea  \
322     13.023438  Smoke Grenade        902        93   
323     20.554688  Smoke Grenade        902        93   
324     14.968750  Smoke Grenade        902        93   
325     18.828125  Smoke Grenade        902        93   
326     11.531250  Smoke Grenade        902        93   
327     19.648438  Smoke Grenade        902        93   
328     12.796875  Smoke Grenade        902        93   
329     14.078125  Smoke Grenade        902        93   
330     11.820312  Smoke Grenade        902        93   

                                               players  \
322  [(143.8098602294922, -1920.9996337890625, -39....   
323  [(189.49594116210938, -1562.5748291015625, -17...   
324  [(-280.8238525390625, -1677.2091064453125, -16...   
325  [(456.9429626464844, -1693.197021484375, -240....   
326  [(64.8831787109375, -2298.50830078125, -39.968...   
327  [(83.26776885986328, -1995.5408935546875, -37....   
328  [(136.1494598388672, -1914.0037841796875, -39....   
329  [(306.8258056640625, -1622.8150634765625, -175...   
330  [(36.8701286315918, -2167.042724609375, -35.96...   

                                throwCords  \
322          (1138.0, -1186.71875, -129.5)   
323        (1138.3125, -1186.75, -129.625)   
324  (1137.71875, -1186.71875, -129.53125)   
325  (1137.15625, -1186.71875, -129.53125)   
326     (1138.0625, -1186.6875, -129.5625)   
327        (1137.6875, -1186.6875, -129.5)   
328   (1137.5625, -1186.71875, -129.53125)   
329   (1138.1875, -1186.65625, -129.46875)   
330      (1136.46875, -1186.75, -129.5625)   

                               landCords  util_score  
322        (-453.71875, -1610.25, -38.0)  759.223187  
323  (-450.0625, -1610.65625, -37.96875)  759.223187  
324      (-452.0, -1607.4375, -37.96875)  759.223187  
325      (-449.53125, -1608.4375, -38.0)  759.223187  
326  (-451.65625, -1604.5625, -37.96875)  759.223187  
327      (-451.1875, -1604.21875, -38.0)  759.223187  
328          (-449.25, -1607.375, -38.0)  759.223187  
329         (-451.8125, -1604.25, -38.0)  759.223187  
330     (-454.25, -1617.0625, -37.96875)  759.223187

In [94]:
df2.to_json('../data/parsed_util.json')